In [1]:
using RigidBodyTreeInspector
using RigidBodyDynamics
import RigidBodyDynamics: edge_to_parent_data
using PyLCM
using BotCoreLCMTypes
import StaticArrays: SVector
import SimpleGradientDescent: NaiveSolver
import DataStructures: OrderedDict
import Flash

adding: /Users/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/DrakeVisualizer/src/lcmtypes to the python path


INFO: Recompiling stale cache file /Users/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/lib/v0.5/EnhancedGJK.ji for module EnhancedGJK.


In [23]:
reload("Flash")

In [24]:
urdf = "$(ENV["HOME"])/locomotion/explorations/point-cloud-signed-distance/examples/data/IRB140/urdf/irb_140_convhull.urdf"
package_path = ["$(ENV["HOME"])/locomotion/drake-distro/drake/examples"]
model = Flash.Models.load_urdf(urdf; package_path=package_path)

Manipulator with 8 links and 7 surfaces

In [25]:
function table()

    mechanism = Mechanism(RigidBody{Float64}("world"))
    parent = root_body(mechanism)
    joint = Joint("joint1", QuaternionFloating{Float64}())
    joint_to_parent = Transform3D(Float64, joint.frameBefore, parent.frame)
    body = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("table_body")))
    body_to_joint = Transform3D(Float64, body.frame, joint.frameAfter)
    attach!(mechanism, parent, joint, joint_to_parent, body, body_to_joint)
    
    width = 0.25
    thickness = 0.05
    surface_points = Vector{SVector{3, Float64}}()
    for z in [-thickness, thickness]
        for x in [-width, width]
            for y in [-width, width]
                push!(surface_points, SVector(x, y, z))
#                 push!(surface_points, Point3D(body.frame, SVector(x, y, z)))
            end
        end
    end
    geometry = SVector{length(surface_points)}(surface_points)
    surfaces = Flash.BodyGeometry[Flash.ConvexGeometry(geometry, body.frame)]
    Flash.Manipulator(mechanism, surfaces)
end
    

table (generic function with 1 method)

In [26]:
model = Flash.Models.load_urdf(urdf; package_path=package_path)
change_joint_type!(model.mechanism, 
    edge_to_parent_data(model.mechanism.toposortedTree[2]), QuaternionFloating{Float64}())

Flash.Models.merge!(model, Flash.Models.squishable())
Flash.Models.merge!(model, table())

Manipulator with 10 links and 9 surfaces

In [27]:
irb_floating_base = edge_to_parent_data(model.mechanism.toposortedTree[findfirst(
    v -> name(v.vertexData.frame) == "base_link", 
    model.mechanism.toposortedTree)])

squishable_floating_base = edge_to_parent_data(model.mechanism.toposortedTree[findfirst(
    v -> name(v.vertexData.frame) == "squishable_body", 
    model.mechanism.toposortedTree)])

table_floating_base = edge_to_parent_data(model.mechanism.toposortedTree[findfirst(
    v -> name(v.vertexData.frame) == "table_body", 
    model.mechanism.toposortedTree)])

Joint "joint1": Quaternion floating joint

In [28]:
state = Flash.ManipulatorState(model);
configuration(state.mechanism_state, squishable_floating_base)[5:7] .= [0.55, 0.45, 0.8]
configuration(state.mechanism_state, irb_floating_base)[5:7] .= [0., 0, .75]
configuration(state.mechanism_state, table_floating_base)[5:7] .= [0.4, 0, 0.6]

# state.mechanism_state.q[end-2:end] = [0.5, 0, 0.25]
setdirty!(state.mechanism_state)
skin = Flash.skin(state);
Visualizer(DrakeVisualizer.contour_mesh(skin, [-.5, -.5, 0.5], [1., .5, 2], 0.01, 0.05))

DrakeVisualizer.Visualizer{Int64}(DataStructures.OrderedDict(1=>DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 4494xFixedSizeArrays.Point{3,Float64},     faces: 8912xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))]),1,PyLCM.LCM(PyObject <LCM object at 0x323009e70>))

In [29]:
Flash.draw(state);

In [30]:
lc = LCM()

PyLCM.LCM(PyObject <LCM object at 0x323009f90>)

In [31]:
# log = PyLCM.pylcm[:EventLog]("/home/rdeits/Desktop/squishable/squishable_squish_out.lcm");
log = PyLCM.pylcm[:EventLog]("../squishable/squishable_squish_out.lcm");

In [47]:
extract_points(msg) = map(SVector{3, Float64}, msg[:points])

extract_points (generic function with 1 method)

In [66]:
function gradient_descent!(state, model, sensed_points)
    solver = NaiveSolver(Flash.num_states(model); rate=0.5, max_step=0.1, iteration_limit=1)
        
    function callback{T}(x::AbstractVector{T}, c)
        x_value = Flash.value.(x)
        current_state = Flash.ManipulatorState(model)
        Flash.GradientDescent.unflatten!(current_state, x_value)
        Flash.draw(current_state, false)
#             skin = Flash.skin(state);
#             Visualizer(DrakeVisualizer.contour_mesh(skin, [-.5, -.5, 0.5], [1., .5, 2], 0.01, 0.05))
    end

    Flash.draw(state, false)
    x_estimated = Flash.flatten(state)
    x_star = Flash.Tracking.estimate_state(model, sensed_points, x_estimated; callback=callback, solver=solver)

    Flash.GradientDescent.unflatten!(state, x_star)
#         skin = Flash.skin(state);
#         Visualizer(DrakeVisualizer.contour_mesh(skin, [-0.5, -.5, 0.4], [1., .5, 2], 0.05, 0.05))
end

gradient_descent! (generic function with 1 method)

In [68]:
msg = PyNULL()
for event in log
    if event[:channel] == "KINECT_POINTS_REDUCED"
        msg = bot_core[:pointcloud_t][:decode](event["data"])
        publish(lc, "DRAKE_POINTCLOUD_kinect", msg)
        sensed_points = map(SVector{3, Float64}, msg[:points][1:200:end])
#         sensed_points = extract_points(msg)[1:200:end]
#         Flash.DepthSensors.draw_points(sensed_points)
        gradient_descent!(state, model, sensed_points)
    end
end
# Profile.clear()
# @profile test_gradient_descent(model, state, sensed_points, 1)
# ProfileView.view()

LoadError: LoadError: InterruptException:
while loading In[68], in expression starting on line 2